In [94]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro, normaltest
import scipy.stats as stats
import itertools
from scipy.stats import ttest_ind
from scipy.stats import f_oneway

In [95]:
df_holidays = pd.read_csv("C:/Users/Aravi/PycharmProjects/CorporaciónFavorita-Ecuador-Sales-Forecasting/data/store-sales-time-series-forecasting/holidays_events.csv")

In [96]:
df_holidays.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [97]:
df_holidays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         350 non-null    object
 1   type         350 non-null    object
 2   locale       350 non-null    object
 3   locale_name  350 non-null    object
 4   description  350 non-null    object
 5   transferred  350 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 14.1+ KB


In [98]:
print("Unique Rows:")
print(df_holidays.isnull().sum())
print("Duplicate Rows:", df_holidays.duplicated().sum())

Unique Rows:
date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64
Duplicate Rows: 0


In [24]:
print("Unique values in 'type':", df_holidays['type'].unique())
print("Unique values in 'locale':", df_holidays['locale'].unique())
print("Unique values in 'locale':", df_holidays['locale_name'].unique())
print("Unique values in 'transferred':", df_holidays['transferred'].unique())

Unique values in 'type': ['Holiday' 'Transfer' 'Additional' 'Bridge' 'Work Day' 'Event']
Unique values in 'locale': ['Local' 'Regional' 'National']
Unique values in 'locale': ['Manta' 'Cotopaxi' 'Cuenca' 'Libertad' 'Riobamba' 'Puyo' 'Guaranda'
 'Imbabura' 'Latacunga' 'Machala' 'Santo Domingo' 'El Carmen' 'Cayambe'
 'Esmeraldas' 'Ecuador' 'Ambato' 'Ibarra' 'Quevedo'
 'Santo Domingo de los Tsachilas' 'Santa Elena' 'Quito' 'Loja' 'Salinas'
 'Guayaquil']
Unique values in 'transferred': [False  True]


In [99]:
df_holidays['date'] = pd.to_datetime(df_holidays['date'])
df_holidays['year'] = df_holidays['date'].dt.year
df_holidays['month'] = df_holidays['date'].dt.month
print("Date Range:", df_holidays['date'].min(), "to", df_holidays['date'].max())

Date Range: 2012-03-02 00:00:00 to 2017-12-26 00:00:00


In [100]:
print(df_holidays.describe())

                                date         year       month
count                            350   350.000000  350.000000
mean   2015-04-24 00:45:15.428571392  2014.714286    7.711429
min              2012-03-02 00:00:00  2012.000000    1.000000
25%              2013-12-23 06:00:00  2013.000000    5.000000
50%              2015-06-08 00:00:00  2015.000000    7.000000
75%              2016-07-03 00:00:00  2016.000000   11.000000
max              2017-12-26 00:00:00  2017.000000   12.000000
std                              NaN     1.614253    3.333366


In [101]:
df_holidays.describe(include=['object','bool'])

,type,locale,locale_name,description,transferred
count,350,350,350,350,350
unique,6,3,24,103,2
top,Holiday,National,Ecuador,Carnaval,False
freq,221,174,174,10,338


In [49]:
holiday_counts = df_holidays.groupby(['year']).size()
print(holiday_counts)

year
2012    41
2013    51
2014    67
2015    52
2016    86
2017    53
dtype: int64


In [102]:
# Count of Holidays and Events per year
holiday_counts = df_holidays.groupby(['year','type']).size().unstack()
print(holiday_counts)

type  Additional  Bridge  Event  Holiday  Transfer  Work Day
year                                                        
2012         7.0     2.0    NaN     31.0       1.0       NaN
2013         9.0     NaN    1.0     38.0       1.0       2.0
2014         9.0     1.0   17.0     38.0       1.0       1.0
2015         8.0     1.0    3.0     39.0       NaN       1.0
2016         9.0     1.0   34.0     38.0       3.0       1.0
2017         9.0     NaN    1.0     37.0       6.0       NaN


In [103]:
# Count of Holidays and Events per year
holiday_counts = df_holidays.groupby(['year','locale']).size().unstack()
print(holiday_counts)

locale  Local  National  Regional
year                             
2012       23        14         4
2013       25        22         4
2014       25        38         4
2015       25        23         4
2016       26        56         4
2017       28        21         4


In [110]:
holiday_counts = df_holidays.groupby(['year','type','locale']).size()
print(holiday_counts)

year  type        locale  
2012  Additional  Local        1
                  National     6
      Bridge      National     2
      Holiday     Local       22
                  National     5
                  Regional     4
      Transfer    National     1
2013  Additional  Local        2
                  National     7
      Event       National     1
      Holiday     Local       23
                  National    11
                  Regional     4
      Transfer    National     1
      Work Day    National     2
2014  Additional  Local        2
                  National     7
      Bridge      National     1
      Event       National    17
      Holiday     Local       23
                  National    11
                  Regional     4
      Transfer    National     1
      Work Day    National     1
2015  Additional  Local        1
                  National     7
      Bridge      National     1
      Event       National     3
      Holiday     Local       24
                

In [109]:
holiday_counts = df_holidays.groupby(['year','type','locale','locale_name']).size()
print(holiday_counts)

year  type        locale    locale_name                   
2012  Additional  Local     Quito                             1
                  National  Ecuador                           6
      Bridge      National  Ecuador                           2
      Holiday     Local     Ambato                            2
                            Cayambe                           1
                                                             ..
2017  Holiday     Regional  Santo Domingo de los Tsachilas    1
      Transfer    Local     Cuenca                            1
                            Ibarra                            1
                            Quito                             1
                  National  Ecuador                           3
Length: 180, dtype: int64


In [108]:
holiday_counts = df_holidays.groupby(['year','transferred','locale']).size()
print(holiday_counts)

year  transferred  locale  
2012  False        Local       23
                   National    13
                   Regional     4
      True         National     1
2013  False        Local       25
                   National    21
                   Regional     4
      True         National     1
2014  False        Local       25
                   National    37
                   Regional     4
      True         National     1
2015  False        Local       25
                   National    23
                   Regional     4
2016  False        Local       25
                   National    54
                   Regional     4
      True         Local        1
                   National     2
2017  False        Local       25
                   National    18
                   Regional     4
      True         Local        3
                   National     3
dtype: int64


In [107]:
holiday_counts = df_holidays.groupby(['year','locale','locale_name','transferred']).size()
print(holiday_counts)

year  locale    locale_name                     transferred
2012  Local     Ambato                          False          2
                Cayambe                         False          1
                Cuenca                          False          1
                El Carmen                       False          1
                Esmeraldas                      False          1
                                                              ..
2017  National  Ecuador                         True           3
      Regional  Cotopaxi                        False          1
                Imbabura                        False          1
                Santa Elena                     False          1
                Santo Domingo de los Tsachilas  False          1
Length: 152, dtype: int64


In [112]:
categorical_cols = ['type', 'locale', 'locale_name', 'transferred']
df_holidays['transferred'] = df_holidays['transferred'].astype(str)
chi_square_results = {}

for col1, col2 in itertools.combinations(categorical_cols, 2):
    contingency_table = pd.crosstab(df_holidays[col1], df_holidays[col2])
    chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
    chi_square_results[f"{col1} vs {col2}"] = {
        'Chi-Square Statistic': chi2,
        'P-value': p,
        'Conclusion': "Significant Difference" if p  < 0.05 else "No Significant Difference",
    }

chi_square_df = pd.DataFrame.from_dict(chi_square_results, orient='index')
chi_square_df

,Chi-Square Statistic,P-value,Conclusion
type vs locale,131.458024,2.354491e-23,Significant Difference
type vs locale_name,180.015456,1.019698e-04,Significant Difference
type vs transferred,7.253206,2.024798e-01,No Significant Difference
locale vs locale_name,700.000000,8.785524e-118,Significant Difference
locale vs transferred,1.861984,3.941625e-01,No Significant Difference
locale_name vs transferred,12.380895,9.641658e-01,No Significant Difference


In [113]:
holiday_groups = [df_holidays[df_holidays['type'] == htype]['year'] for htype in df_holidays['type'].unique()]
f_stat, p_value = f_oneway(*holiday_groups)
print(f"ANOVA Test (Year vs Holiday Type) → P-Value: {p_value}")
if p_value < 0.05:
   print("Conclusion: Significant Difference")

ANOVA Test (Year vs Holiday Type) → P-Value: 0.004846774954971729
Conclusion: Significant Difference
